# GPT-dev

Notebook for developing first steps into building GPT from scratch.

### Imports

In [23]:
import os
import torch

ModuleNotFoundError: No module named 'torch'

### Data Loading

Downloading Tiny Sheakespeare Dataset for a light version training of GPT:

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


--2023-10-11 01:43:07--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1,1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1,06M  --.-KB/s    in 0,05s   

2023-10-11 01:43:08 (23,3 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
if not os.path.exists("../data"):
    os.makedirs("../data")

In [4]:
!mv input.txt ../data/

### Data Inspection

In [5]:
with open("../data/input.txt", "r", encoding="utf-8") as f:
    text = f.read()

Datset length in characters:

In [6]:
len(text)

1115394

First 200 characters:

In [11]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


Unique characters:

In [12]:
chars = sorted(list(set(text)))
len(chars)

65

In [16]:
"".join(chars)

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

### Text Tokenization

Translate characters to integers in a simple manner:
> OpenAI uses [tiktoken](https://github.com/openai/tiktoken)

In [19]:
str_to_int_dict = {ch:i for i,ch in enumerate(chars)}
int_to_str_dict = {i:ch for i,ch in enumerate(chars)}

def encode(string):
    return [str_to_int_dict[char] for char in string]

def decode(list_of_int):
    return "".join([int_to_str_dict[i] for i in list_of_int])

In [21]:
encode("hello world")

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]

In [22]:
decode([46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42])

'hello world'

In [24]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

NameError: name 'torch' is not defined

First 100 encoded characters:

In [25]:
data[:100]

NameError: name 'data' is not defined

### Data Split

Train and test sets (90/10%):

In [26]:
n = int(0.9*len(data))

train_data = data[:n]
test_data = data[n:]

NameError: name 'data' is not defined

### Data Loader

Generate batches of chunks of data:

In [27]:
#Example
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

NameError: name 'train_data' is not defined

In [29]:
torch.manual_seed(77)

batch_size = 4
block_size = 8

def get_batch(split: str):
    
    if split == "train":
        data = train_data
    elif split == "val":
        data = val_data
    else:
        return
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

NameError: name 'torch' is not defined

In [30]:
xb, yb = get_batch("train")

print("Inputs:", xb.shape)
print(xb)

print("Outputs:", yb.shape)
print(yb)

NameError: name 'get_batch' is not defined

Input-target pairs:

In [31]:
for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"When input is {context.tolist()} the target: {target}")

NameError: name 'batch_size' is not defined